In [344]:
class rayleigh_vapor:
    """Generates a vapor dataset from interpolated Rayleigh data"""
    def __init__(self,vdc_name=None,varnames=None,varfiles=None, rayleigh_root=None, 
                vapor_root=None, nxyz=None, grid_file=None, force=False):
        self.numts=len(varfiles)
        self.varnames=varnames
        self.vdc_name=vdc_name+'.vdc'
        self.vdc_data_dir = vdc_name+'_data'
        self.nvars=len(varnames)
        self.varfiles=varfiles
        varstring=' '
        for i in range(self.nvars):
            varstring=varstring+varnames[i]
            if (i != self.nvars-1):
                varstring=varstring+':'
        self.varstring=varstring
        print(self.varstring)
        print(self.numts)
        self.vapor_root=vapor_root
        self.rayleigh_root=rayleigh_root
        self.nxyz=nxyz
        self.grid_file=grid_file
        if (force == True):
            print('Parameter "force" is set to true.')
            print('Removing: '+self.vdc_name+' > /dev/null')
            print('Removing: '+self.vdc_data_dir+' > /dev/null')
            self.destroy_vdc()            
    def create_vdc(self, force=False):
        import subprocess as sp
        res=str(self.nxyz)
        cube_string = res+'x'+res+'x'+res
        cmd1 = 'export PATH=$PATH:'+self.vapor_root+'/bin'
        cmd2 = ' && export LD_LIBRARY_PATH='+self.vapor_root+'/lib:$LD_LIBRARY_PATH'
        cmd3 = ' && vdccreate -dimension '+cube_string+' -numts '+str(self.numts)
        cmd3 = cmd3+' -vars3d '+self.varstring+' '+self.vdc_name
        creation_cmd=cmd1+cmd2+cmd3+' && ls'
        print(creation_cmd)
        s=sp.Popen(creation_cmd,shell=True)
        #b=s.read()
        #print(b)
    def populate_vdc(self):
        for i in range(self.numts):
            for j in range(self.nvars):
                infile=self.varfiles[i][j]
                ofile=infile+'.cube'
                self.rayleigh_to_cube(infile,ofile)
                self.cube_to_vdc(ofile,i,j)
    def rayleigh_to_cube(self,infile,ofile):
        import subprocess as sp
        cmd1 = 'export PATH=$PATH:'+self.rayleigh_root
        cmd2 = ' && echo $PATH && interp3d -i '+infile+' -o '+ofile+' -g '+self.grid_file+' -N '+str(self.nxyz)
        cmd = cmd1+cmd2
        s=sp.Popen(cmd,shell=True)
        #b=s.read()
        #print(b)
    def cube_to_vdc(self,ofile,timeind,varind):
        import subprocess as sp
        cmd1 = 'export PATH=$PATH:'+self.vapor_root+'/bin'
        cmd2 = ' && export LD_LIBRARY_PATH='+self.vapor_root+'/lib:$LD_LIBRARY_PATH'
        cmd3 = ' && raw2vdc -ts '+str(timeind)+' -varname '+self.varnames[varind]
        cmd3 = cmd3+' '+self.vdc_name+' '+ofile
        cmd = cmd1+cmd2+cmd3
        s=sp.Popen(cmd, shell=True)
        #b=s.read()
        #print(b)
        #print(cmd)
    def destroy_vdc(self):
        import subprocess as sp
        cmd1 = 'rm -rf '+self.vdc_name +' > /dev/null'
        cmd2 = 'rm -rf '+self.vdc_data_dir+' > /dev/null'
        #s = sp.Popen(cmd1, shell=True)
        #print(s.communicate())
        try:
            s=sp.Popen(cmd1,shell=True)
            s.wait(timeout=10)
            print(cmd1)
        except:
            print('cmd1 error', cmd1)
            s.communicate()
            pass
        #os.popen.wait()
        print(cmd2)
        try:
            s=sp.Popen(cmd2,shell=True)
            s.wait(timeout=120)
            print(cmd2)
        except:
            print('cmd2 error', cmd2)
        s= sp.Popen('ls -ltr', shell=True)
        #os.popen.wait()
                

In [345]:
f = ['02086101_0001', '02086101_2807']
gf = '02086101_grid'
files = [f]
vdc = 'jup_test'
varnames = ['vr','vortz']
vroot='/custom/software/VAPOR3-3.2.0-Linux'
rroot='/home/feathern/devel/forks/Nick/Rayleigh/post_processing/interpolation'
cube_size=256
test = rayleigh_vapor(vdc_name=vdc,varnames=varnames,varfiles=files,vapor_root=vroot,
                       nxyz=cube_size, grid_file=gf, rayleigh_root=rroot, force=True)
test.create_vdc(force=True)

 vr:vortz
1
Parameter "force" is set to true.
Removing: jup_test.vdc > /dev/null
Removing: jup_test_data > /dev/null
rm -rf jup_test.vdc > /dev/null
rm -rf jup_test_data > /dev/null
rm -rf jup_test_data > /dev/null
export PATH=$PATH:/custom/software/VAPOR3-3.2.0-Linux/bin && export LD_LIBRARY_PATH=/custom/software/VAPOR3-3.2.0-Linux/lib:$LD_LIBRARY_PATH && vdccreate -dimension 256x256x256 -numts 1 -vars3d  vr:vortz jup_test.vdc && ls


In [346]:
test.populate_vdc()